In [2]:
import numpy as np
import matplotlib.pyplot as plt 
import network
from numba import jit, double, int8, int64
from scipy.ndimage import uniform_filter1d

# Aufgabe 4 
> Zielsetzung:
- Faltung bei konstanter Temperatur $T = 1$
- Auftragung der von Energie über Zeit (Monte-Carlo-Schritte)
- Struktur der resultierenden, gefalteten Zustände
- Abschätzung der Energiebarriere

> Durchführung der Monte-Carlo-Simulation: 

In [2]:
@jit((double[:])(network.Protein.class_type.instance_type, int8, int64))
def monte_carlo_const_temperature(protein, temperature, number_of_steps):
    
    simulation = np.zeros(number_of_steps, dtype=double)
    simulation[0] = protein.energy
    for step in range(1, number_of_steps):
        protein.random_fold_step(temperature)
        simulation[step] = protein.energy
    
    return simulation

steps = 1e9
temperature = 1

p = network.create_protein(interaction_type='normal')
arr = monte_carlo_const_temperature(p, temperature, steps)


> Auffinden und Mittelung der Energiebarrieren + Auftragung:

In [ ]:
@jit((double[:])(int64, double[:]))
def energyBounds(number_of_steps, simulation):

    energy = np.zeros(number_of_steps, dtype=double)
    min_energy = 0
    last_energy = 0
    threshold = 0.3
    last_peak = False
    
    for i in range(number_of_steps):
        current_energy = simulation[i]
        
        if current_energy <= min_energy - 1:
            if last_peak:
                energy[i] = abs(last_energy - min_energy)
                
                
            min_energy = current_energy
            last_peak = False
            
        if current_energy > min_energy:
            last_peak = True
            
        last_energy = current_energy
    energy = energy[energy > threshold + 1]
    return energy[energy!=0] 

fig, ax = plt.subplots()

ax.set_xlabel('Monte-Carlo-Schritte')
ax.set_ylabel('Energie in a.u.')
ax.set_xscale('log')
plt.grid()

bounds = energyBounds(steps, arr)
label =''
if len(bounds) > 0:
    label = r'$\langle \Delta E \rangle$ = '+str(round(np.mean(bounds), 2))+' a.u.'
ax.plot(np.arange(steps), arr, color='blue', label=label)

if len(bounds) > 0:
    plt.legend(loc='lower left')

> Zeichnung der Proteinkette:

In [3]:
network.plot_protein(p)

NameError: name 'p' is not defined